## 📀 Локальное подключение Google Диска

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls

drive  sample_data


## 🔮 Информация о GPU

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, 
and then re-execute this cell.


## 📁 Читаем датасет

In [ ]:
import pandas as pd


df = pd.read_csv("drive/My Drive/lenta-ru-news.csv")

# df = pd.read_csv("/content/drive/MyDrive/Datasets/data_from_Sait/lenta-ru-news.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
df.head()

,url,title,text,topic,tags,date
0,https://lenta.ru/news/1914/09/16/hungarnn/,1914. Русские войска вступили в пределы Венгрии,Бои у Сопоцкина и Друскеник закончились отступ...,Библиотека,Первая мировая,1914/09/16
1,https://lenta.ru/news/1914/09/16/lermontov/,1914. Празднование столетия М.Ю. Лермонтова от...,"Министерство народного просвещения, в виду про...",Библиотека,Первая мировая,1914/09/16
2,https://lenta.ru/news/1914/09/17/nesteroff/,1914. Das ist Nesteroff!,"Штабс-капитан П. Н. Нестеров на днях, увидев в...",Библиотека,Первая мировая,1914/09/17
3,https://lenta.ru/news/1914/09/17/bulldogn/,1914. Бульдог-гонец под Льежем,Фотограф-корреспондент Daily Mirror рассказыва...,Библиотека,Первая мировая,1914/09/17
4,https://lenta.ru/news/1914/09/18/zver/,1914. Под Люблином пойман швабский зверь,"Лица, приехавшие в Варшаву из Люблина, передаю...",Библиотека,Первая мировая,1914/09/18


## 🎨 Выбираем темы

In [ ]:
topics_list = df['topic'].unique() # получаем список тем

topics_lengths = [len(df['text'][(df['topic']==t)].reset_index(drop=True)) \
                  for t in topics_list] # длины тем

print("🎨 Тема - 🎰 Количество документов в теме:\n")
for t, len_t in zip(topics_list, topics_lengths):
  print('{} - {}'.format(t, len_t))

🎨 Тема - 🎰 Количество документов в теме:

Библиотека - 65
Россия - 160445
Мир - 136621
Экономика - 79528
Интернет и СМИ - 44663
Спорт - 64413
Культура - 53797
Из жизни - 27605
Силовые структуры - 19596
Наука и техника - 53136
Бывший СССР - 53402
nan - 0
Дом - 21734
Сочи - 1
ЧМ-2014 - 2
Путешествия - 6408
Ценности - 7766
Легпром - 114
Бизнес - 7399
МедНовости - 1
Оружие - 3
69-я параллель - 1268
Культпросвет  - 340
Крым - 666


In [ ]:
list(df['tags'][(df['topic']=='Спорт')].reset_index(drop=True).unique())

['Все',
 nan,
 'Летние виды',
 'Бокс и ММА',
 'Футбол',
 'Зимние виды',
 'Хоккей',
 'Авто']

In [ ]:
tmp_df = df[df['topic']=='Спорт']
len(tmp_df[tmp_df['tags']=='Футбол'])

13128

In [ ]:
# Составляем словарь тема - теги 
topics_tags_dict = {}
for t in topics_list:
  topics_tags_dict[t] = list(df['tags'][(df['topic']==t)].reset_index(drop=True).unique())

print("🎨 Тема - 🔷 Тег -  🎰 Количество документов в теге:\n")
topics_tags_list = []
for t in topics_list:
  tmp_df = df[df['topic']==t]
  for tag in topics_tags_dict[t]:
    l = len(tmp_df[tmp_df['tags']==tag])
    topics_tags_list.append((t, tag, l))

from operator import itemgetter
topics_tags_list = sorted(topics_tags_list, key=itemgetter(2), reverse=True)

for x in topics_tags_list:
  print(x)

🎨 Тема - 🔷 Тег -  🎰 Количество документов в теге:

('Россия', 'Все', 111887)
('Мир', 'Все', 95126)
('Экономика', 'Все', 55252)
('Спорт', 'Все', 36697)
('Наука и техника', 'Все', 34537)
('Культура', 'Все', 34351)
('Бывший СССР', 'Все', 28266)
('Интернет и СМИ', 'Все', 27210)
('Мир', 'Политика', 18513)
('Россия', 'Общество', 18237)
('Бывший СССР', 'Украина', 17934)
('Из жизни', 'Все', 15413)
('Россия', 'Политика', 14707)
('Дом', 'Все', 13861)
('Спорт', 'Футбол', 13128)
('Экономика', 'Госэкономика', 12869)
('Мир', 'Общество', 9323)
('Культура', 'Кино', 9109)
('Интернет и СМИ', 'Интернет', 7945)
('Наука и техника', 'Наука', 6975)
('Силовые структуры', 'Следствие и суд', 6825)
('Россия', 'Происшествия', 6225)
('Культура', 'Музыка', 5823)
('Мир', 'Происшествия', 5754)
('Из жизни', 'Люди', 5010)
('Мир', 'Преступность', 4737)
('Экономика', 'Бизнес', 4264)
('Бизнес', 'Бизнес', 3960)
('Дом', 'Квартира', 3759)
('Наука и техника', 'Космос', 3622)
('Мир', 'Конфликты', 3136)
('Интернет и СМИ', 'ТВ и

Выберем темы, которые наиболее отличаются между собой и при этом содержат достаточное количество документов:


1.   Россия
2.   Экономика
3.   Спорт
4.   Культура
5.   Наука и техника
6.   Путешествия
7.   Силовые структуры

Составим словарь из этих тем. В каждую тему возьмем 6400 документов, так как в теме *Путешествия* всего 6408 документов.




In [ ]:
# list_selecting_topics = [
#                          'Россия',
#                          'Экономика',
#                          'Спорт',
#                          'Культура',
#                          'Наука и техника',
#                          'Путешествия',
#                          'Силовые структуры'
#                          ]

In [ ]:
list_selecting_topics = [
                         ('Наука и техника', 'Техника'),
                         ('Культура', 'Книги'),
                         ('Наука и техника', 'Оружие'),
                         ('Культура', 'Театр'),
                         ('Силовые структуры', 'Полиция и спецслужбы'),
                         ('Наука и техника', 'Гаджеты'),
                         ('Спорт', 'Бокс и ММА'),
                         ('Наука и техника', 'Космос'),
                         ('Наука и техника', 'Наука'),
                         ('Мир', 'Политика'),
                         ('Экономика', 'Госэкономика')
]

topics_dict = {}
for t in list_selecting_topics:
  topic, tag = t[0], t[1]
  tmp_df = df[df['topic']==topic]
  tmp_df = tmp_df[tmp_df['tags']==tag]
  topics_dict[topic + '-' + tag] = list(tmp_df['text'].reset_index(drop=True)[:1000])

In [ ]:
topics_dict.keys()

dict_keys(['Спорт-Хоккей', 'Наука и техника-Техника', 'Культура-Книги', 'Наука и техника-Оружие', 'Культура-Театр', 'Силовые структуры-Полиция и спецслужбы', 'Наука и техника-Игры', 'Наука и техника-Гаджеты', 'Из жизни-Звери', 'Спорт-Бокс и ММА', 'Силовые структуры-Криминал', 'Наука и техника-Космос', 'Культура-Музыка', 'Наука и техника-Наука', 'Мир-Политика', 'Экономика-Госэкономика'])

In [ ]:
topics_dict['Наука и техника-Оружие'][29]

'Группа ученых рассказала о последствиях, которые могли бы произойти в результате подводных испытаний водородной бомбы КНДР. Об этом пишет Motherboard. В четверг, 21 сентября, министр иностранных дел КНДР Ли Ён Хо (Ri\xa0Yong-ho) объявил, что глава государства Ким Чен Ын рассматривает возможность испытаний водородной бомбы в Тихом океане. Такой ответ последовал в результате угроз президента США Дональда Трампа «полностью уничтожить Северную Корею». Издание опросило специалистов, которые рассказали о возможных последствиях такого шага. Профессор Нью-Йоркского университета Оливер Бюлер рассказал, что в результате детонации образуются большие волны. Это повлечет за собой подъем на поверхность огромного количества водяного пара и различного мусора. Вместе с этим ударная волна и радиация убьет практически все живое в непосредственной близости. К примеру, после испытания ядерной бомбы в 1946 году из воды достали около 38 тысяч мертвых рыб. При этом волны не смогут создать цунами, обнадеживае